### Creación del dataset

In [ ]:
using CSV
using DataFrames

include("../src/data.jl")
dataset_folder = "../resources/csv_files"
dataset = CreateDataset(dataset_folder)
if !isfile("../resources/dataset.csv")
    CSV.write("../resources/dataset.csv", dataset)
end;

### Summary

In [ ]:
n_ins, n_vars = size(dataset)
println("Número de variables: ", n_vars)
println("Número de instancias: ", n_ins)
println("Número de individuos: ", length(unique(dataset[:, 1])))
println("Número de clases: ", length(unique(dataset[:, end])))

### Gestión de ausentess

In [ ]:
print(data)